# First Attempt

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
import numpy as np
import fnmatch
import scipy.io as sio

In [87]:
import tensorflow as tf

In [5]:
PATH = "data\\unlabelled"
sz=224

In [6]:
os.listdir(PATH)

['0.20 s.tifheightmap.mat',
 '0.78 s.tifheightmap.mat',
 '0.79 s.tifheightmap.mat',
 '0.80 s.tifheightmap.mat',
 '12.59 s.tifheightmap.mat',
 '12.60 s.tifheightmap.mat',
 '14.80 s.tifheightmap.mat',
 '16.80 s.tifheightmap.mat',
 '19.60 s.tifheightmap.mat',
 '22.00 s.tifheightmap.mat',
 '27.00 s.tifheightmap.mat',
 '28.79 s.tifheightmap.mat',
 '30.00 s.tifheightmap.mat',
 '32.00 s.tifheightmap.mat',
 '33.40 s.tifheightmap.mat',
 '36.19 s.tifheightmap.mat',
 '36.20 s.tifheightmap.mat',
 '38.20 s.tifheightmap.mat',
 '43.40 s.tifheightmap.mat',
 '44.60 s.tifheightmap.mat',
 '47.59 s.tifheightmap.mat',
 '47.60 s.tifheightmap.mat',
 '51.80 s.tifheightmap.mat',
 '52.60 s.tifheightmap.mat',
 '53.60 s.tifheightmap.mat',
 '54.40 s.tifheightmap.mat',
 '55.00 s.tifheightmap.mat',
 '56.00 s.tifheightmap.mat',
 '56.59 s.tifheightmap.mat',
 '57.00 s.tifheightmap.mat',
 '59.60 s.tifheightmap.mat',
 '6.80 s.tifheightmap.mat',
 '8.78 s.tifheightmap.mat',
 '8.79 s.tifheightmap.mat',
 '8.80 s.tifheightmap

In [83]:
files = []

x = 0
y = 0

filenames = []
hm_list = []

for file in os.listdir(PATH):
    if fnmatch.fnmatch(file, '*.mat'):
        files.append(file)
        hm = sio.loadmat(f'{PATH}\\{file}')['height_r']
        # print(hm.shape[0])
        hm_list.append(hm)
        filenames.append(file)
        
        x=max(hm.shape[0],x)
        y=max(hm.shape[1],y)
        
print(x)
print(y)

hm_array = []

for m in hm_list:
    hm = np.pad(np.copy(m), ((0,x-m.shape[0]),(0,y-m.shape[1])), 'constant', constant_values=(0))
    hm_array.append(hm)

df = pd.DataFrame(hm_list[0])
df.to_csv("hm.csv")    

df = pd.DataFrame(hm_np[0,:,:])
df.to_csv("hm_np.csv")

hm_np = np.array(hm_array)
print(hm_np.shape)
print(hm_list[0])
print(hm_np[0,:,:])

584
368
(35, 584, 368)
[[0.29372 0.34945 0.38375 ... 0.      0.      0.     ]
 [0.34831 0.41226 0.45008 ... 0.      0.      0.     ]
 [0.39016 0.45942 0.49873 ... 0.      0.      0.     ]
 ...
 [0.      0.      0.      ... 0.24495 0.20439 0.15776]
 [0.      0.      0.      ... 0.2131  0.1794  0.13998]
 [0.      0.      0.      ... 0.17431 0.14779 0.11631]]
[[0.29372 0.34945 0.38375 ... 0.      0.      0.     ]
 [0.34831 0.41226 0.45008 ... 0.      0.      0.     ]
 [0.39016 0.45942 0.49873 ... 0.      0.      0.     ]
 ...
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]]


In [73]:
# img = plt.imread(f'{PATH}\\{files[0]}')
# plt.imshow(img);

In [18]:
hm = sio.loadmat(f'{PATH}\\{files[0]}')['height_r']

In [19]:
hm

array([[0.29372, 0.34945, 0.38375, ..., 0.     , 0.     , 0.     ],
       [0.34831, 0.41226, 0.45008, ..., 0.     , 0.     , 0.     ],
       [0.39016, 0.45942, 0.49873, ..., 0.     , 0.     , 0.     ],
       ...,
       [0.     , 0.     , 0.     , ..., 0.24495, 0.20439, 0.15776],
       [0.     , 0.     , 0.     , ..., 0.2131 , 0.1794 , 0.13998],
       [0.     , 0.     , 0.     , ..., 0.17431, 0.14779, 0.11631]])

## Variational Auto-Encoder

In [84]:
learning_rate = 0.001
# Input and target placeholders
inputs_ = tf.placeholder(tf.float32, (None, x,y,1), name="input")
targets_ = tf.placeholder(tf.float32, (None, x,y,1), name="target")

### Encoder
conv1 = tf.layers.conv2d(inputs=inputs_, filters=16, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 584x368x16
maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), padding='same')
# Now 292x184x16
conv2 = tf.layers.conv2d(inputs=maxpool1, filters=8, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 292x184x8
maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), padding='same')
# Now 146x92x8
conv3 = tf.layers.conv2d(inputs=maxpool2, filters=8, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 146x92x8
encoded = tf.layers.max_pooling2d(conv3, pool_size=(2,2), strides=(2,2), padding='same')
# Now 73x46x8

### Decoder
upsample1 = tf.image.resize_images(encoded, size=(146,92), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 7x7x8
conv4 = tf.layers.conv2d(inputs=upsample1, filters=8, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
upsample2 = tf.image.resize_images(conv4, size=(292,184), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 14x14x8
conv5 = tf.layers.conv2d(inputs=upsample2, filters=8, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
upsample3 = tf.image.resize_images(conv5, size=(584,368), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# Now 28x28x8
conv6 = tf.layers.conv2d(inputs=upsample3, filters=16, kernel_size=(3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16

logits = tf.layers.conv2d(inputs=conv6, filters=1, kernel_size=(3,3), padding='same', activation=None)
#Now 28x28x1

# Pass logits through sigmoid to get reconstructed image
decoded = tf.nn.sigmoid(logits)

# Pass logits through sigmoid and calculate the cross-entropy loss
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)

# Get cost and define the optimizer
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Training

In [89]:
sess = tf.Session()

In [97]:
epochs = 20
batch_size = 5
shuffle_size = 1000
sess.run(tf.global_variables_initializer())

ds = tf.data.Dataset.from_tensor_slices(hm_np).shuffle(shuffle_size).batch(batch_size).repeat()

iterator = ds.make_one_shot_iterator()
imgs = iterator.get_next()

for e in range(epochs):
    for ii in range(hm_np.shape[0]//batch_size):
        # imgs = batch[0].reshape((-1, x, y, 1))
        batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_: imgs,
                                                         targets_: imgs})

        print("Epoch: {}/{}...".format(e+1, epochs),
              "Training loss: {:.4f}".format(batch_cost))
        

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.

## K-Means

## PCA